In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
import numpy as np
from tensorboardX import SummaryWriter
torch.manual_seed(1)

In [2]:
USE_CUDA = torch.cuda.is_available() # gpu 사용

### Weight Decay 데이터

In [3]:
BATCH_SIZE = 4

In [4]:
class CustomDataset(torchdata.Dataset):
    def __init__(self):
        
        data = np.loadtxt('./weight_decay_data.csv',
                        delimiter=',', dtype=np.float32)
        
        self.length = data.shape[0]
        self.x = torch.Tensor(data[:,:2])
        self.y = torch.Tensor(data[:,2]).unsqueeze(1)
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.length 

In [5]:
train_dataset = CustomDataset()

train_loader = torchdata.DataLoader(dataset=train_dataset,
                                                      batch_size=BATCH_SIZE, 
                                                      shuffle=True,
                                                      num_workers=2)

## Model 

In [6]:
class NN(nn.Module):
    def __init__(self,hidden_size):
        super(NN,self).__init__()
        self.l1 = nn.Linear(2,hidden_size)
        self.l2 = nn.Linear(hidden_size,1)
    
    def forward(self,inputs):
        outputs = F.relu(self.l1(inputs))
        return F.sigmoid(self.l2(outputs))

## Learning rate scheduling 

### 1. StepLR 

In [7]:
LR = 0.1
model = NN(6)
optimizer = optim.SGD(model.parameters(),lr=LR)
loss_function = nn.BCELoss()

In [8]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
for epoch in range(100):
    
    losses=[]
    for i,(inputs,targets) in enumerate(train_loader):
        model.zero_grad()
        pred = model(Variable(inputs))
        loss = loss_function(pred,Variable(targets))
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    if epoch % 30 ==0:
        print("epoch : %3d, mean_loss : %.3f, lr : %.4f " % (epoch,np.mean(losses),scheduler.get_lr()[0]))

epoch :   0, mean_loss : 0.584, lr : 0.1000 
epoch :  30, mean_loss : 0.383, lr : 0.0100 
epoch :  60, mean_loss : 0.313, lr : 0.0010 
epoch :  90, mean_loss : 0.309, lr : 0.0001 


### 2. MultiStepLR 

In [9]:
LR = 0.1
model = NN(6)
optimizer = optim.SGD(model.parameters(),lr=LR)
loss_function = nn.BCELoss()

In [10]:
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
for epoch in range(100):
    
    losses=[]
    for i,(inputs,targets) in enumerate(train_loader):
        model.zero_grad()
        pred = model(Variable(inputs))
        loss = loss_function(pred,Variable(targets))
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    if epoch % 10 ==0:
        print("epoch : %3d, mean_loss : %.3f, lr : %.4f " % (epoch,np.mean(losses),scheduler.get_lr()[0]))

epoch :   0, mean_loss : 0.637, lr : 0.1000 
epoch :  10, mean_loss : 0.375, lr : 0.1000 
epoch :  20, mean_loss : 0.313, lr : 0.1000 
epoch :  30, mean_loss : 0.342, lr : 0.0100 
epoch :  40, mean_loss : 0.317, lr : 0.0100 
epoch :  50, mean_loss : 0.317, lr : 0.0100 
epoch :  60, mean_loss : 0.317, lr : 0.0100 
epoch :  70, mean_loss : 0.316, lr : 0.0100 
epoch :  80, mean_loss : 0.313, lr : 0.0010 
epoch :  90, mean_loss : 0.316, lr : 0.0010 
